# Natural Policy Gradient (NPG)

as in [01_Simplest_Policy_Gradient_Implementations.ipynb](01_Simplest_Policy_Gradient_Implementations.ipynb)

we want to perform gradient ascent in model's parameter space to maximize policy's performance $J(\pi_{\theta_t})$

_(remember that $J$ is expected rewards over all possible trajectories)_

So the update rule is
$$\theta_{t+1} = \theta_t + \alpha \nabla_{\theta} J(\pi_{\theta_t}) = \theta_t + \alpha \vec{g}$$
where step direction 
$$\boxed{\vec{g} = \nabla_{\theta} J(\pi_{\theta}) =  \frac{1}{|D|} \sum_{\tau \in D}R(\tau)\cdot\sum_{t=0}^{T} \nabla_{\theta} \ log  \ \pi_\theta(a_t|s_t) }$$
Where $R(\tau)$ is a reward for a particular trajectory $\tau$. Check different variants in a linked notebook.

## Broad idea and similarity to 2nd order optimization
NPG improves ['Vanilla Policy Gradient'](01_Simplest_Policy_Gradient_Implementations.ipynb) in a similar (in spirit) way that Second Order optimization methods improve upon a First Order ([Notes_Second_Order_Methods.ipynb](../../optimization/Notes_Second_Order_Methods.ipynb)).

Specifically, descent method is augmented with information about second order derivatives (curvature).

Lets remind us that in 2nd order optimization we use Taylor expansion and search for its minima $\vec{x}^*$.    
$$f(\vec{x}) \approx f(\vec{x}_0) + \nabla f(\vec{x}_0) \cdot (\vec{x} - \vec{x}_0) + \frac{1}{2} (\vec{x} - \vec{x}_0)\cdot H(\vec{x}_0) (\vec{x} - \vec{x}_0)$$

$$\nabla f(\vec{x}^*) = \nabla f(\vec{x}_0) + H(\vec{x}_0) (\vec{x}^*- \vec{x}_0) = \vec{0}$$

$$\vec{x}^* = \vec{x}_0 + H(\vec{x}_0)^{-1}  \nabla f(\vec{x}_0)$$

Here a _Hessian matrix_ $H$ provides information about the curvature of objective function.

_Intuition: Curvature - rate of change of slope, or slope on steroids xd. Hessian is inverted, since if curvature is low -  we are on plateau, we want to scale steps larger (1/small = big). For high curvature we want an opposite effect._

It is an elegant method to adapt step size, but it is computationally expensive.
***

For NPG we also want to employ similar construct to control gradient a/-descent. 
### Role of KL divergence
Small policy's parameter change (step size) makes policy learning more stable. _This idea also plays important role in offline learning/importance sampling, which will be discussed in TRPO method._

This 'smooth' trajectory though parameter space can be enforced by keeping old policy 'similar enough' to new policy.

Differences between two policies (probability distributions) can be measured using KL divergence. [KL_Divergence.ipynb](../../Statistics/KL_Divergence.ipynb)
***
### Optimization problem
Optimization goal for each iteration could be to find such step:
$$\delta^* = \underset{s.t. \ D_{KL}\big(\pi(\theta)||\pi(\theta+\delta)\big) \lt \epsilon}{\argmax_\delta} J(\theta + \delta)$$
_(https://www.andrew.cmu.edu/course/10-703/slides/Lecture_NaturalPolicyGradientsTRPOPPO.pdf)_

We can soften constrain by forming an _unconstrained_ objective function

$$\delta^* = \argmax_\delta J(\theta + \delta) - \lambda \bigg(D_{KL}\big(\pi(\theta)||\pi(\theta+\delta)\big)- \epsilon\bigg)$$

_term $D_{KL}\big(\pi(\theta)||\pi(\theta+\delta)\big) \geq 0$ (since probabilities are non-negative) and KL term can act only as a penalty._
### KL Divergence and Fisher information
KL divergence is related to Fisher Information (Matrix) [Fisher_Information.ipynb (ending)](../../Statistics/Fisher_Information.ipynb)

$$\boxed{D_{KL}\bigg(\pi(x; \theta) ||\pi(x; \theta + \delta) \bigg) \overset{\text{2nd order}}{\approx} \frac{1}{2}\delta^T \mathbb{E}\bigg[\nabla_\theta \log \pi(x; \theta) \ \nabla_\theta \log \pi(x; \theta)^T\bigg] \delta}$$
Where expectation is Fisher information $F$ (or $I(\theta)$)
$$F = \mathbb{E}\bigg[\nabla_\theta \log \pi(x; \theta) \ \nabla_\theta \log \pi(x; \theta)^T\bigg]$$
and perturbation $\delta$ is difference between new and old policy (paramter) variants
$$\delta = \theta_{new} - \theta_{old}$$

***
### Finding optimization direction via curvature
We approximate $J(\theta + \delta)$ up to 2nd order via Taylor's expansion and get

$$\delta^* = \argmax_\delta J(\theta) + \nabla_\theta J(\theta)\big|_{\theta = \theta_{old}}\delta - \lambda D_{KL}\big(\pi(\theta)||\pi(\theta+\delta)\big)+ \lambda \epsilon$$
$$ = \argmax_\delta  \nabla_\theta J(\theta)_{old}\delta - \frac{\lambda}{2}\delta^T F|_{\theta = \theta_{old}} \delta \underbrace{+ \lambda \epsilon + J(\theta)}_{\text{not important for optimization}}$$
Our objective function is 
$$f = \nabla_\theta J(\theta)_{old}\delta - \frac{\lambda}{2}\delta^T F|_{\theta = \theta_{old}} \delta$$
_We can change maximization task into minimization by multiplying objective by -1._

Searching for extremum we compute a derivative and set it to zero:
$$\frac{\partial}{\partial \delta} \bigg(\nabla_\theta J(\theta)_{old}\delta - \frac{\lambda}{2}\delta^T F|_{\theta = \theta_{old}} \delta\bigg) = 0$$
and we get
$$0 = \nabla_\theta J(\theta)_{old} - \frac{\lambda}{2}F|_{\theta = \theta_{old}} \delta^*$$
$$\frac{\lambda}{2}F|_{\theta = \theta_{old}} \delta^* = \nabla_\theta J(\theta)_{old} $$
$$ \delta^* = \frac{2}{\lambda} F^{-1}\nabla_\theta J(\theta) $$
Which is a modified original step direction/size.

By absorbing constants into $\alpha$ our iteration update is
$$\theta_{t+1} = \theta_t + \alpha  F^{-1}\nabla_\theta J(\theta)$$

***
### Finding step size via KL threshold value $\epsilon$
Yet we still have unknown step size $\alpha$ and KL divergence threshold has be discarded during optimization.

We can retrieve step size via "normalization under the Fisher metric":
$$D_{KL}\bigg(\pi(x; \theta) ||\pi(x; \theta + \delta) \bigg) = \frac{1}{2}\delta^{*T} F \delta^* \leq \epsilon$$
$$\frac{1}{2}\delta^{*T} F \delta^* = \frac{1}{2}\bigg(\alpha  F^{-1}\nabla_\theta J(\theta)\bigg)^T F \bigg(\alpha  F^{-1}\nabla_\theta J(\theta)\bigg) \leq \epsilon$$
$$ = \frac{\alpha^2}{2} \nabla_\theta J(\theta)^T {F^{-1}}^T \cancel{F F^{-1}}\nabla_\theta J(\theta)\leq \epsilon$$
Fisher information matrix, like Hessian, is symmetric, due to second derivatives.
$$\frac{\alpha^2}{2} \nabla_\theta J(\theta)^T {F^{-1}}\nabla_\theta J(\theta)\leq \epsilon$$
At maximum ('$\leq$' $\longrightarrow$ '=' ), $\alpha$ should be 
$$\boxed{\alpha = \sqrt{\frac{2\epsilon}{\nabla J^T {F^{-1}}\nabla J}}}$$

## Final expression
If we return to short notation for gradient
$$\vec{g} = \nabla_\theta J(\theta)$$
update rule is the following
$$\theta_{t+1} = \theta_t + \sqrt{\frac{2\epsilon}{\vec{g}^T {F^{-1}} \vec{g}}} F^{-1}\vec{g}$$

## For future (see TRPO)
One can see that we use inverse of a matrix $F^{-1}\vec{g}$, which is costly to compute.

One might say that it is a solution to a problem
$$F\vec{x} = \vec{g} \rightarrow \vec{x} = F^{-1}\vec{g}$$
Then one can compute $\vec{x}$ only approximately, which will be much faster. But you will see it it TRPO method description.